### 0. 환경 세팅

In [24]:
from dotenv import load_dotenv
from rich import print as rprint


load_dotenv()

True

# Context Engineering
핵심 철학 : 에이전트가 실패하는 이유는 성능 보단, 올바른 `컨텍스트`가 제공되지 않았기 때문
> 그래서 <br> LLM이 작업을 수행할 수 있도록 올바른 정보와 도구를 올바른 형식으로 제공하는 것이 목표!



1. **사용자 입력** 📝
2. **LLM 호출** 🤖
    - 응답 생성
    - 도구 호출 여부 결정
3. **도구 실행** 🔧 (필요시)
4. **반복** 🔄 (원하는 결과가 나올 때까지)

**Context의 5가지 유형**

1. **Instruction** - 명령어 및 지시사항
2. **Tool** - 도구 및 기능
3. **Session** - 세션 정보
4. **Long-term Memory** - 장기 메모리
5. **Runtime Configuration Context** - 런타임 설정 컨텍스트

## 1. System Prompt

> ```@dynamic_prompt```<br>
상황에 맞춰 System prompt만 바꿀 수 있음

### Runtime Context
> 사용자 ID에 맞춰서 변화

In [21]:
# 기본 에이전트

from langchain.agents import create_agent

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # middleware=[],
    # context_schema=""
    debug=True,
)
response = agent.invoke({"messages": "안녕하세요"})
rprint(response)

[values] {'messages': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}, id='2c7f7f26-05a7-4de0-b86c-c00448121ba8')]}
[updates] {'model': {'messages': [AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--ca418c7f-9039-4367-b187-8517cf4c0a15-0', usage_metadata={'input_tokens': 2, 'output_tokens': 8, 'total_tokens': 10, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}, id='2c7f7f26-05a7-4de0-b86c-c00448121ba8'), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider'

{
    'messages': [
        HumanMessage(
            content='안녕하세요',
            additional_kwargs={},
            response_metadata={},
            id='2c7f7f26-05a7-4de0-b86c-c00448121ba8'
        ),
        AIMessage(
            content='안녕하세요! 무엇을 도와드릴까요?',
            additional_kwargs={},
            response_metadata={
                'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.5-flash-lite',
                'safety_ratings': [],
                'model_provider': 'google_genai'
            },
            id='lc_run--ca418c7f-9039-4367-b187-8517cf4c0a15-0',
            usage_metadata={
                'input_tokens': 2,
                'output_tokens': 8,
                'total_tokens': 10,
                'input_token_details': {'cache_read': 0}
            }
        )
    ]
}

In [28]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

# 1. Runtime Context: 외부에서 주입받을 설정값 틀
@dataclass
class AnimalContext:
    role: str = "" # dog or cat

# 2. 동적 시스템 프롬프트 로직 -> Middleware에 적용
@dynamic_prompt
def animal_persona_prompt(request: ModelRequest) -> str:
    # 실행 시점에 주입된 Runtime Context 값을 활용.
    char_role = request.runtime.context.role

    if char_role == "dog":
        return "당신은 강아지입니다. 말 끝마다 '멍'을 붙여서 쾌할하게 말하세요."
    elif char_role == "cat":
        return "당신은 고양이입니다. 귀찮은 말투로 말 끝마다 '냥'을 붙이세요."
    
    return "친절하게 답하세요."

# 3. 에이전트 생성
agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[],
    middleware=[animal_persona_prompt], # 위에서 만든 프롬프트 로직 적용
    context_schema=AnimalContext        # 어떤 데이터를 받을지 스키마 등록
)


#### 오류 발생지점!

In [32]:
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "안녕 방가워~"
    }]},
    # context=AnimalContext(role="") # 이 부분 빠지면 오류남, 빈 객체라도 넣어줘야 함!
)
rprint(response)

# 이게 또, 미들웨어 설정을 해주면 그냥 해서는 안되네

AttributeError: 'NoneType' object has no attribute 'role'

In [27]:
# 4. 에이전트 실행
response_dog = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "안녕 방가워~"
    }]},
    context=AnimalContext(role="dog")
)

response_cat = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "안녕 방가워~"
    }]},
    context=AnimalContext(role="cat")
)

response  = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "안녕 방가워~"
    }]},
    context=AnimalContext(role="")
)

rprint(response_dog)
rprint(response_cat)
rprint(response)

{
    'messages': [
        HumanMessage(
            content='안녕 방가워~',
            additional_kwargs={},
            response_metadata={},
            id='6d487a94-b703-4e8e-9bcf-2441c1ab04c9'
        ),
        AIMessage(
            content='안녕! 만나서 반가워 멍! 😊',
            additional_kwargs={},
            response_metadata={
                'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.5-flash-lite',
                'safety_ratings': [],
                'model_provider': 'google_genai'
            },
            id='lc_run--203fd965-afcf-4aa7-841e-b2880be1a8a4-0',
            usage_metadata={
                'input_tokens': 32,
                'output_tokens': 12,
                'total_tokens': 44,
                'input_token_details': {'cache_read': 0}
            }
        )
    ]
}

{
    'messages': [
        HumanMessage(
            content='안녕 방가워~',
            additional_kwargs={},
            response_metadata={},
            id='e5823e18-cf48-44b5-978f-33e9e9e04b8e'
        ),
        AIMessage(
            content='안녕, 반가워 냥. 뭘 도와줄까 냥?',
            additional_kwargs={},
            response_metadata={
                'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.5-flash-lite',
                'safety_ratings': [],
                'model_provider': 'google_genai'
            },
            id='lc_run--96ad4ed6-741d-4508-9d69-c8f81f4508b9-0',
            usage_metadata={
                'input_tokens': 30,
                'output_tokens': 16,
                'total_tokens': 46,
                'input_token_details': {'cache_read': 0}
            }
        )
    ]
}

{
    'messages': [
        HumanMessage(
            content='안녕 방가워~',
            additional_kwargs={},
            response_metadata={},
            id='e434f20f-c66a-4abc-8383-73f90262ae16'
        ),
        AIMessage(
            content='안녕하세요! 만나서 반갑습니다. 😊\n\n무엇을 도와드릴까요? 궁금한 점이 있으시면 언제든지 편하게
물어보세요.',
            additional_kwargs={},
            response_metadata={
                'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.5-flash-lite',
                'safety_ratings': [],
                'model_provider': 'google_genai'
            },
            id='lc_run--7e63b992-490a-42b3-9df6-bb647e9cae5f-0',
            usage_metadata={
                'input_tokens': 13,
                'output_tokens': 30,
                'total_tokens': 43,
                'input_token_details': {'cache_read': 0}
            }
        )
    ]
}

### Store
> 장기 메모리에서 사용자 선호도에 맞춰서 변화
---
이거는 맞춤형으로 할 때 생각보다 괜찮을 듯, 아직은 사용 x

State 사용법은 필요성 x

## 2. Messages
> LLM에 전송되는 prompt 구성

---
> `@wrap_model_call`<br>
LLM을 호출하는 과정 전체를 감싸고(Wrap) 있어서,<br>
호출 전/후로 뭐든 가능
<br>
<br>- 모델 교체
<br>- 툴 조작
<br>- 판단
<br>- 결과 수정

### 모델 라우팅 미들웨어

In [21]:
from langchain.chat_models import init_chat_model
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
from langchain.agents import create_agent

# 1. 모델 준비
fast_model = init_chat_model("google_genai:gemini-2.5-flash-lite")
model = init_chat_model("google_genai:gemini-2.5-flash")
pro_model = init_chat_model("google_genai:gemini-2.5-pro")

@wrap_model_call
def intelligent_switcher(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    
    # 1. 사용자의 마지막 메시지 가져오기
    last_message = request.messages[-1].content
    
    # 2. 판단 단계 : LLM에게 의도 분류 요청
    judge_prompt = f"""
    다음 사용자의 메시지를 분석해서 'CODING', 'COOKING', 'GENERAL' 중 하나로만 분류해서 답해.
    다른 말은 하지 말고 단어 하나만 딱 뱉어.

    사용자 메시지: {last_message}
    """

    # 판단용 모델 호출
    category = fast_model.invoke(judge_prompt).content.strip().upper()
    print(f"[{category}] 모드로 전환.")

    # 3. 설정 번경 : 판단 결과에 따라 프롬프트 & 모델 교체
    new_system_prompt = "당신은 친절한 어시스턴트입니다. 간결히 답하세요."
    new_model = model

    if category == "CODING":
        new_system_prompt = "당신은 프로그래밍 전문가입니다. 사용자의 문제를 해결하는 코드를 작성해주세요."
        new_model = pro_model
    elif category == "COOKING":
        new_system_prompt = "당신은 요리 전문가입니다. 사용자의 요리 레시피를 찾아주세요."
        new_model = model
    else:
        new_system_prompt = "당신은 친절한 어시스턴트입니다. 간결히 답하세요. 이모지를 많이 쓰세요."
        new_model = fast_model

    # 4. 변경된 내용 적용
    updated_message = [
        {"role": "system", "content": new_system_prompt},
        *request.messages
    ]    

    request = request.override(
        messages=updated_message,
        model=new_model
    )

    # 5. 변경된 요청 처리
    return handler(request)
    

agent = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[],
    middleware=[intelligent_switcher],
)

In [22]:
code_response = agent.invoke({
    "messages": [{
        "role": "user",
        "content":  "리스트 컴프리헨션 예제 좀 만들어줘."
    }]
})

cook_response = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "된장찌개 맛있게 만드는 방법 알려줘."
    }]
})

normal_response = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "파이썬이 뭐임, 뱀임?"
    }]
})

[CODING] 모드로 전환.
[COOKING] 모드로 전환.
[CODING] 모드로 전환.


In [25]:
rprint(code_response)
rprint(cook_response)
rprint(normal_response)

{
    'messages': [
        HumanMessage(
            content='리스트 컴프리헨션 예제 좀 만들어줘.',
            additional_kwargs={},
            response_metadata={},
            id='2acfaf12-01a7-4f36-aac5-9f8f7adfcfec'
        ),
        AIMessage(
            content='네, 프로그래밍 전문가로서 파이썬의 강력하고 우아한 기능인 **리스트 컴프리헨션(List 
Comprehension)** 예제를 단계별로, 이해하기 쉽게 만들어 드리겠습니다.\n\n리스트 컴프리헨션은 기존의 리스트나 다른 
순회 가능한(iterable) 객체를 기반으로 새로운 리스트를 간결하게 만드는 방법입니다. 일반적인 `for` 루프보다 코드가 
짧고 가독성이 높아 많이 사용됩니다.\n\n### 기본 구조\n\n```python\n[표현식 for 항목 in 
반복가능한_객체]\n```\n\n---\n\n### Level 1: 가장 기본적인 사용법\n\n전통적인 `for` 루프와 비교하며 
살펴보겠습니다.\n\n**예제 1: 0부터 9까지의 숫자로 리스트 만들기**\n\n```python\n# 전통적인 for 루프 방식\nsquares =
[]\nfor i in range(10):\n    squares.append(i)\nprint(f"For 루프: {squares}")\n\n# 리스트 컴프리헨션 
방식\nsquares_comp = [i for i in range(10)]\nprint(f"컴프리헨션: {squares_comp}")\n```\n**결과:**\n```\nFor 루프: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]\n컴프리헨션: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]\n```\n\n**예제 2: 1부터 5까지의 숫자를 
제곱한 리스트 만들기**\n\n```python\n# 전통적인 for 루프 방식\nsquares = []\nfor x in range(1, 6):\n    
squares.append(x**2)\nprint(f"For 루프: {squares}")\n\n# 리스트 컴프리헨션 방식\nsquares_comp = [x**2 for x in 
range(1, 6)]\nprint(f"컴프리헨션: {squares_comp}")\n```\n**결과:**\n```\nFor 루프: [1, 4, 9, 16, 25]\n컴프리헨션: 
[1, 4, 9, 16, 25]\n```\n\n---\n\n### Level 2: 조건문(`if`) 추가하기\n\n리스트를 만들 때 특정 조건을 만족하는 항목만
포함시킬 수 있습니다.\n\n```python\n[표현식 for 항목 in 반복가능한_객체 if 조건문]\n```\n\n**예제 3: 0부터 9까지의 
숫자 중 짝수만 포함하는 리스트 만들기**\n\n```python\n# 전통적인 for 루프 방식\nevens = []\nfor i in range(10):\n  
if i % 2 == 0:\n        evens.append(i)\nprint(f"For 루프: {evens}")\n\n# 리스트 컴프리헨션 방식\nevens_comp = [i 
for i in range(10) if i % 2 == 0]\nprint(f"컴프리헨션: {evens_comp}")\n```\n**결과:**\n```\nFor 루프: [0, 2, 4, 6, 
8]\n컴프리헨션: [0, 2, 4, 6, 8]\n```\n\n**예제 4: 주어진 리스트에서 길이가 5 이상인 단어만 대문자로 
바꾸기**\n\n```python\nwords = [\'apple\', \'banana\', \'cherry\', \'kiwi\', \'orange\', \'grape\']\n\n# 전통적인 
for 루프 방식\nlong_words = []\nfor word in words:\n    if len(word) >= 5:\n        
long_words.append(word.upper())\nprint(f"For 루프: {long_words}")\n\n# 리스트 컴프리헨션 방식\nlong_words_comp = 
[word.upper() for word in words if len(word) >= 5]\nprint(f"컴프리헨션: 
{long_words_comp}")\n```\n**결과:**\n```\nFor 루프: [\'APPLE\', \'BANANA\', \'CHERRY\', \'ORANGE\', 
\'GRAPE\']\n컴프리헨션: [\'APPLE\', \'BANANA\', \'CHERRY\', \'ORANGE\', \'GRAPE\']\n```\n\n---\n\n### Level 3: 
조건부 표현식(`if-else`) 사용하기\n\n`if` 뿐만 아니라 `else`까지 사용하여 조건에 따라 다른 값을 리스트에 추가할 수 
있습니다.\n**주의: `if-else`는 `for` 앞에 위치합니다.**\n\n```python\n[참일때_표현식 if 조건문 else 거짓일때_표현식
for 항목 in 반복가능한_객체]\n```\n\n**예제 5: 0부터 9까지 숫자 중 짝수면 \'짝수\', 홀수면 \'홀수\'를 리스트에 
담기**\n\n```python\n# 전통적인 for 루프 방식\nlabels = []\nfor i in range(10):\n    if i % 2 == 0:\n        
labels.append(\'짝수\')\n    else:\n        labels.append(\'홀수\')\nprint(f"For 루프: {labels}")\n\n# 리스트 
컴프리헨션 방식\nlabels_comp = [\'짝수\' if i % 2 == 0 else \'홀수\' for i in range(10)]\nprint(f"컴프리헨션: 
{labels_comp}")\n```\n**결과:**\n```\nFor 루프: [\'짝수\', \'홀수\', \'짝수\', \'홀수\', \'짝수\', \'홀수\', 
\'짝수\', \'홀수\', \'짝수\', \'홀수\']\n컴프리헨션: [\'짝수\', \'홀수\', \'짝수\', \'홀수\', \'짝수\', \'홀수\', 
\'짝수\', \'홀수\', \'짝수\', \'홀수\']\n```\n\n---\n\n### Level 4: 중첩 루프(Nested Loops) 사용하기\n\n`for` 
루프를 여러 개 사용하여 다차원 리스트를 처리하거나 복잡한 조합을 만들 수 있습니다.\n\n```python\n[표현식 for 
외부항목 in 외부객체 for 내부항목 in 내부객체]\n```\n\n**예제 6: 2차원 리스트를 1차원 리스트로 만들기 
(Flatten)**\n\n```python\nmatrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]\n\n# 전통적인 for 루프 방식\nflattened = 
[]\nfor row in matrix:\n    for num in row:\n        flattened.append(num)\nprint(f"For 루프: {flattened}")\n\n# 
리스트 컴프리헨션 방식\nflattened_comp = [num for row in matrix for num in row]\nprint(f"컴프리헨션: 
{flattened_comp}")\n```\n**결과:**\n```\nFor 루프: [1, 2, 3, 4, 5, 6, 7, 8, 9]\n컴프리헨션: [1, 2, 3, 4, 5, 6, 7, 
8, 9]\n```\n\n**예제 7: 구구단 2단, 3단 만들기**\n\n```python\n# 전통적인 for 루프 방식\ngugudan = []\nfor i in 
range(2, 4):\n    for j in range(1, 10):\n        gugudan.append(f"{i} x {j} = {i*j}")\n# print(gugudan) # 너무 
길어서 일부만 출력\n\n# 리스트 컴프리헨션 방식\ngugudan_comp = [f"

{
    'messages': [
        HumanMessage(
            content='된장찌개 맛있게 만드는 방법 알려줘.',
            additional_kwargs={},
            response_metadata={},
            id='d38fa4c3-06dc-4c7b-ae1f-9147a4c6f4c5'
        ),
        AIMessage(
            content="안녕하세요! 한국인의 소울푸드, 구수하고 깊은 맛의 된장찌개는 정말 언제 먹어도 질리지 않는 
메뉴죠. 요리 전문가로서, 단순히 재료를 넣는 것을 넘어 '정말 맛있는' 된장찌개를 만드는 비법을 자세히 
알려드릴게요.\n\n이 레시피는 기본적인 된장찌개에 깊은 맛을 더하는 팁들을 포함하고 있습니다.\n\n---\n\n### **✨ 
맛있는 된장찌개의 핵심 비법! ✨**\n\n1.  **육수는 기본 중의 기본:** 맹물 대신 멸치 다시마 육수를 사용하면 깊은 
감칠맛이 차원이 달라집니다.\n2.  **된장 선택과 활용:** 재래된장과 시판 된장을 섞어 쓰면 풍미가 더욱 풍부해져요. 
(재래된장 7: 시판된장 3 비율 추천)\n3.  **쌀뜨물 활용:** 쌀뜨물은 된장찌개의 맛을 부드럽게 하고 구수한 맛을 
더해줍니다. 육수와 섞어 사용해도 좋아요.\n4.  **재료의 순서:** 단단하고 익히는 데 시간이 걸리는 재료부터 넣고, 
나중에 부드러운 재료를 넣어 익힘 정도를 맞춰줍니다.\n5.  **마늘과 고춧가루는 나중에:** 처음부터 넣기보다 재료가 
어느 정도 익은 후 넣어 향을 살려줍니다.\n\n---\n\n### **🍚 구수하고 깊은 맛의 된장찌개 레시피 
🥘**\n\n**[재료]**\n\n*   **필수 재료:**\n    *   된장 3~4큰술 (재래된장 + 시판된장 섞어서 사용하면 좋아요)\n    * 
두부 1/2모 (부침용 또는 찌개용)\n    *   애호박 1/4개\n    *   양파 1/4개\n    *   감자 1/2개\n    *   대파 1/2대\n
*   청양고추 1개 (선택 사항, 매운맛 조절)\n    *   홍고추 1/2개 (선택 사항, 색감용)\n    *   다진 마늘 1/2큰술\n   
*   고춧가루 1/2큰술 (선택 사항)\n    *   식용유 약간 (선택 사항, 재료 볶을 때 사용)\n*   **육수 재료 (필수!):**\n 
*   물 700ml (또는 쌀뜨물)\n    *   국물용 멸치 7~10마리\n    *   다시마 5x5cm 2장\n*   **선택 추가 재료 (더욱 
풍성한 맛을 위해):**\n    *   바지락 또는 표고버섯, 느타리버섯, 팽이버섯\n    *   차돌박이 또는 돼지고기 (목살, 
삼겹살 등)\n    *   무 한 조각 (시원한 맛 추가)\n\n**[재료 손질]**\n\n1.  **두부:** 1.5~2cm 두께로 먹기 좋게 
깍둑썰기 합니다.\n2.  **애호박, 감자, 양파:** 두부와 비슷한 크기로 깍둑썰기 합니다. 감자는 물에 잠시 담가 전분기를 
제거하면 더욱 깔끔합니다.\n3.  **대파, 청양고추, 홍고추:** 어슷썰기 합니다.\n4.  **멸치:** 머리와 내장을 제거하여 
비린 맛을 줄입니다. (선택 사항이지만 추천)\n5.  **선택 재료 손질:** 바지락은 해감하고, 버섯류는 먹기 좋게 찢거나 
썰고, 고기는 한입 크기로 썰어 준비합니다.\n\n**[만드는 방법]**\n\n1.  **육수 내기:**\n    *   냄비에 물(또는 
쌀뜨물) 700ml와 손질한 멸치, 다시마를 넣고 끓입니다.\n    *   물이 끓기 시작하면 다시마는 5분 뒤 건져내고, 멸치는 
중불에서 10~15분 더 끓여 진한 육수를 만듭니다. 멸치도 건져냅니다.\n    *   (Tip: 쌀뜨물과 육수를 섞어 사용하면 더욱
구수해요!)\n\n2.  **재료 볶기 (선택 사항, 깊은 맛 추가):**\n    *   뚝배기 또는 냄비에 식용유를 약간 두르고 양파를 
먼저 볶아 향을 냅니다.\n    *   차돌박이나 돼지고기를 넣을 경우, 양파와 함께 볶아 고기의 맛을 끌어올립니다. 고기가 
어느 정도 익으면 된장 1큰술을 넣고 함께 볶아 구수한 향을 더합니다.\n\n3.  **된장 풀기 & 재료 넣기:**\n    *   
만들어 둔 육수에 된장을 체에 걸러 곱게 풀어줍니다. (체에 걸러야 찌개가 깔끔하고 덩어리가 지지 않아요.)\n    *   
감자와 무(넣는다면)를 먼저 넣고 끓입니다. (단단한 재료부터 익혀야 해요.)\n\n4.  **나머지 재료 넣기:**\n    *   
감자가 반쯤 익으면 애호박, 버섯류, 바지락(넣는다면)을 넣고 끓입니다.\n    *   이때 다진 마늘과 고춧가루(선택 
사항)를 넣어줍니다.\n\n5.  **두부와 마무리:**\n    *   재료들이 거의 다 익고 국물이 보글보글 끓어오르면 두부, 대파,
청양고추, 홍고추를 넣고 한소끔 더 끓여줍니다.\n    *   (Tip: 불을 끄기 직전에 두부를 넣어야 두부가 너무 물러지지 
않고 부드럽습니다.)\n\n6.  **맛 조절:**\n    *   간을 보고 싱거우면 된장을 조금 더 풀거나 국간장 1/2작은술을 
추가합니다.\n    *   너무 짜면 육수나 물을 약간 더 넣습니다.\n    *   (Tip: 마지막에 설탕 1/3작은술 정도를 넣으면 
된장의 쓴맛을 잡고 감칠맛을 더할 수 있어요.)\n\n---\n\n**[더 맛있게 즐기는 팁]**\n\n*   **뚝배기에 끓여 드세요:** 
뚝배기는 온기를 오래 유지하여 마지막 한 숟갈까지 따뜻하고 맛있게 즐길 수 있습니다.\n*   **밥에 비벼 먹기:** 따뜻한 
밥에 된장찌개 건더기와 국물을 듬뿍 넣고 슥슥 비벼 먹으면 밥도둑이 따로 없죠!\n*   **청국장 섞기:** 된장과 청국장을 
섞어 끓이면 더욱 깊고 구수한 풍미를 느낄 수 있습니다.\n\n---\n\n이 레시피대로 만드시면 분명 온 가족이 엄지척할 만한
맛있는 된장찌개를 맛보실 수 있을 거예요. 맛있게 만들어 드시고 즐거운 식사 시간 되세요! 😊",
            additional_kwargs={},
            response_metadata={
                'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.5-flash',
                'safety_ratings': [],
                'model_provider': 'google_genai'
            },
            id='lc_run--2b009eab-4b2b-4957-8c91-19b7fe0f2ef9-0',
            usage_metadata={
                'input_tokens': 32,
                'output_tokens': 3012,
                'total_tokens': 3044,
                'input_token_details': {'cache_read': 0},
                'output_token_details': {'reasoning': 1364}
            }
        )
    ]
}

{
    'messages': [
        HumanMessage(
            content='파이썬이 뭐임, 뱀임?',
            additional_kwargs={},
            response_metadata={},
            id='853f062b-b1b7-4fec-b70d-f5ab4b6e4865'
        ),
        AIMessage(
            content='하하, 재미있는 질문이네요! 아주 좋은 지적입니다.\n\n결론부터 말씀드리면, 질문하신 
**파이썬(Python)은 뱀이 아니라 프로그래밍 언어**가 맞습니다.\n\n하지만 \'파이썬\'이라는 단어 자체는 원래 아프리카나
아시아에 사는 \'비단뱀\'을 뜻하는 단어입니다. 프로그래밍 언어의 이름이 왜 뱀 이름이 되었는지, 그리고 이 언어가 
정확히 무엇인지 전문가답게 설명해 드릴게요.\n\n### 파이썬(Python), 넌 누구냐?\n\n#### 1. 이름의 유래: 뱀이 아니라 
코미디!\n\n프로그래밍 언어 파이썬을 만든 귀도 반 로섬(Guido van Rossum)이라는 개발자는 1970년대 영국 BBC에서 
방영했던 코미디 쇼, **"몬티 파이튼의 날아다니는 서커스(Monty Python\'s Flying Circus)"**의 열렬한 팬이었습니다. 
그래서 새로 만든 언어의 이름을 이 코미디 그룹의 이름에서 따와 \'파이썬\'이라고 지었습니다. 뱀과는 직접적인 관련은 
없는 셈이죠. 😄\n\n#### 2. 그래서 파이썬이 뭔데?\n\n간단히 말해, **사람이 컴퓨터와 대화하기 위해 사용하는 
\'언어\'** 중 하나입니다.\n\n우리가 한국어나 영어를 사용해 다른 사람과 소통하는 것처럼, 개발자들은 파이썬이라는 
언어를 사용해서 컴퓨터에게 "이런 프로그램을 만들어줘", "이 데이터를 분석해줘" 와 같은 명령을 내립니다.\n\n#### 3. 
왜 이렇게 인기가 많을까?\n\n파이썬은 현재 전 세계적으로 가장 인기 있는 프로그래밍 언어 중 하나입니다. 그 이유는 
명확합니다.\n\n*   **문법이 아주 쉽고 간결합니다.** 사람이 쓰는 영어와 비슷해서 처음 프로그래밍을 배우는 사람도 
금방 익숙해질 수 있습니다.\n*   **\'만능 언어\'입니다.** 웹사이트 개발, 인공지능(AI), 데이터 분석, 게임 제작, 업무 
자동화 등 거의 모든 분야에서 사용할 수 있습니다.\n*   **강력한 \'레고 블록\'들이 많습니다.** 전 세계 개발자들이 
미리 만들어 둔 수많은 기능(라이브러리)들이 있어서, 복잡한 기능도 몇 줄의 코드로 쉽게 가져와 사용할 수 
있습니다.\n\n### 파이썬으로 딱 한 줄만 코딩해볼까요?\n\n파이썬이 얼마나 쉬운지 보여드리기 위해, 화면에 "안녕, 
파이썬!"이라고 출력하는 코드를 보여드릴게요.\n\n```python\nprint("안녕, 파이썬!")\n```\n\n**이게 끝입니다.** 정말 
간단하죠?\n\n참고로, 다른 언어(예: Java)로 똑같은 작업을 하려면 보통 아래와 같이 더 복잡한 코드를 작성해야 
합니다.\n\n```java\npublic class HelloWorld {\n    public static void main(String[] args) {\n        
System.out.println("안녕, 파이썬!");\n    }\n}\n```\n\n파이썬의 간결함이 한눈에 보이실 겁니다.\n\n### 그래서 이걸로
뭘 할 수 있는데?\n\n우리가 일상에서 사용하는 많은 서비스들이 파이썬으로 만들어졌습니다.\n\n*   **유튜브, 
인스타그램**: 초기 개발에 파이썬이 핵심적인 역할을 했습니다.\n*   **넷플릭스, 멜론**: 사용자의 취향을 분석해서 
콘텐츠를 추천하는 시스템에 파이썬이 널리 쓰입니다.\n*   **알파고**: 이세돌 9단과 대결했던 인공지능 알파고의 핵심 
로직도 파이썬으로 만들어졌습니다.\n*   **엑셀 정리, 파일 자동 분류**: 매일 반복하는 지루한 사무 업무를 자동화하는 
프로그램을 쉽게 만들 수 있습니다.\n\n---\n\n### **요약**\n\n*   **파이썬은 뱀인가?** -> 아니요, **프로그래밍 
언어**입니다.\n*   **이름은 왜 파이썬인가?** -> 개발자가 좋아하던 **영국 코미디 그룹** 이름에서 따왔습니다.\n*   
**무엇에 쓰는 물건인가?** -> 컴퓨터에게 명령을 내리는 **대화 수단**입니다.\n*   **특징은?** -> **아주 쉽고, 
강력하며, 어디에나 쓸 수 있는** 최고의 입문용 언어입니다.\n\n혹시 프로그래밍에 첫발을 내딛고 싶으시다면, 파이썬은 
정말 훌륭한 선택이 될 겁니다. 더 궁금한 점이 있으시면 언제든지 다시 물어보세요',
            additional_kwargs={},
            response_metadata={
                'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.5-pro',
                'safety_ratings': [],
                'model_provider': 'google_genai'
            },
            id='lc_run--9b427f67-10d8-452b-a2b6-48d6175a5df1-0',
            usage_metadata={
                'input_tokens': 31,
                'output_tokens': 2632,
                'total_tokens': 2663,
                'input_token_details': {'cache_read': 0},
                'output_token_details': {'reasoning': 1615}
            }
        )
    ]
}